<a href="https://colab.research.google.com/github/Murphy-Mary/MyProject/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

souce :https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis
!unzip /content/twitter-entity-sentiment-analysis.zip && rm /content/twitter-entity-sentiment-analysis.zip

Mounted at /content/gdrive
100% 1.99M/1.99M [00:01<00:00, 2.19MB/s]
100% 1.99M/1.99M [00:01<00:00, 1.92MB/s]
Archive:  /content/twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [ ]:
!pip install -q -U --pre pycaret
!python -m venv .env
!source /content/.env/bin
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect]'
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.1/480.1 KB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 6.2 MB/s eta 0:0

In [ ]:
import numpy as np
import pandas as pd
import spacy
from tqdm.auto import tqdm
import time
from sklearn.metrics import accuracy_score

In [ ]:
print(spacy.prefer_gpu())

True


## data

In [ ]:
df = pd.read_csv('/content/twitter_training.csv',header=None)

In [ ]:
df = df.rename(columns={0: "Id",1:"entity",2:"sentiment",3:"Tweet_content"})

In [ ]:
df

,Id,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


# EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             74682 non-null  int64 
 1   entity         74682 non-null  object
 2   sentiment      74682 non-null  object
 3   Tweet_content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
df.isna().sum()

Id                 0
entity             0
sentiment          0
Tweet_content    686
dtype: int64

In [ ]:
 df = df[df['Tweet_content'].notna()]

In [ ]:
df.isnull().sum()

Id               0
entity           0
sentiment        0
Tweet_content    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73996 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             73996 non-null  int64 
 1   entity         73996 non-null  object
 2   sentiment      73996 non-null  object
 3   Tweet_content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


In [ ]:
df.duplicated().sum()

2340

In [ ]:
df.drop_duplicates(inplace=True)

/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.loc[df["sentiment"] == "Negative",'new_sent'] = 0
df.loc[df["sentiment"] == "Positive",'new_sent'] = 1
df.loc[df["sentiment"] == "Neutral",'new_sent'] = 2
df.loc[df["sentiment"] == "Irrelevant",'new_sent'] = 3
# df.loc[df["sentiment"] != "Neutral" and df["sentiment"] != "Positive" and df["sentiment"] != "Negative",'new_sent'] = 3

/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
df.isna().sum()

Id               0
entity           0
sentiment        0
Tweet_content    0
new_sent         0
dtype: int64

In [ ]:
df['new_sent'] = df['new_sent'].fillna(4)

<ipython-input-18-3fcc4110a6d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_sent'] = df['new_sent'].fillna(4)


In [ ]:
df

,Id,entity,sentiment,Tweet_content,new_sent
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,1.0
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,1.0
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,1.0
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,1.0
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,1.0
...,...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,1.0
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,1.0
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,1.0
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,1.0


#prepare

In [ ]:
# Load the English NLP model
spacy.prefer_gpu() # or spacy.require_gpu()
nlp = spacy.load('en_core_web_sm')

# Define stop words
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
def preprocess_text(text):
    # Parse the text with Spacy
    doc = nlp(text)
    
    # Lemmatize the tokens and remove stop words
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop]
    
    # Join the lemmas back into a string and return it
    return " ".join(lemmas)

In [ ]:
%%time

tqdm.pandas()

df['Tweet_content'] = df['Tweet_content'].progress_apply(preprocess_text)

  0%|          | 0/71656 [00:00<?, ?it/s]

CPU times: user 23min 2s, sys: 9.06 s, total: 23min 11s
Wall time: 23min 53s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df

,Id,entity,sentiment,Tweet_content,new_sent
0,2401,Borderlands,Positive,"m get borderland murder ,",1.0
1,2401,Borderlands,Positive,"come border kill ,",1.0
2,2401,Borderlands,Positive,"m get borderland kill ,",1.0
3,2401,Borderlands,Positive,"m come borderland murder ,",1.0
4,2401,Borderlands,Positive,"m get borderland 2 murder ,",1.0
...,...,...,...,...,...
74677,9200,Nvidia,Positive,realize windows partition mac like 6 year nvid...,1.0
74678,9200,Nvidia,Positive,realize mac window partition 6 year nvidia dri...,1.0
74679,9200,Nvidia,Positive,realize window partition mac 6 year nvidia dri...,1.0
74680,9200,Nvidia,Positive,realize window partition mac like 6 year nvidi...,1.0


# Vectorizing the Text Data

In [ ]:
from sklearn.model_selection import train_test_split

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Tweet_content'], df['new_sent'], test_size=0.2, random_state=248362023)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# create bag-of-words features
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# ML


## Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# train a Naive Bayes classifier
clf = MultinomialNB()
clf

In [ ]:
clf.fit(X_train_vect, y_train)

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=300, random_state = 746563)
clf

RandomForestClassifier(n_estimators=300, random_state=746563)

In [ ]:
clf.fit(X_train_vect, y_train)

RandomForestClassifier(n_estimators=300, random_state=746563)

## neural_network 

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(300,), random_state=746563)
#5,2
#10,2
#10,5

In [ ]:
clf.fit(X_train_vect, y_train)

# Predictions Section

In [ ]:
# evaluate the classifier on the test set
y_pred = clf.predict(X_test_vect)

In [ ]:
y_pred

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

## Save model

In [ ]:
import pickle

filename = "RDF3.pickle"
# pickle.dump(clf, open(filename, "wb"))
with open('/content/gdrive/MyDrive/Models/Twitter/'+ filename , 'wb') as f:
    pickle.dump(clf, f)
# loaded_model = pickle.load(open(filename, "rb"))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True)

# Classification Report

In [ ]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['Negative', 'Positive','Neutral','Irrelevant']
print(classification_report(y_test, y_pred, target_names=target_names))


nlp +NN ML Hiden= 200
```
              precision    recall  f1-score   support

    Negative       0.91      0.88      0.90      4297
    Positive       0.84      0.90      0.87      4009
     Neutral       0.89      0.87      0.88      3592
  Irrelevant       0.90      0.85      0.87      2434

    accuracy                           0.88     14332
   macro avg       0.88      0.88      0.88     14332
weighted avg       0.88      0.88      0.88     14332
```



RDF ML 
n = 10

```
              precision    recall  f1-score   support

    Negative       0.88      0.91      0.89      4297
    Positive       0.82      0.92      0.86      4009
     Neutral       0.92      0.85      0.88      3592
       other       0.94      0.81      0.87      2434

    accuracy                           0.88     14332
   macro avg       0.89      0.87      0.88     14332
weighted avg       0.88      0.88      0.88     14332

```
n = 100 not set seed


```
                precision    recall  f1-score   support

    Negative       0.91      0.92      0.92      4297
    Positive       0.85      0.93      0.89      4009
     Neutral       0.94      0.88      0.91      3592
       other       0.96      0.85      0.90      2434

    accuracy                           0.90     14332
   macro avg       0.91      0.90      0.90     14332
weighted avg       0.91      0.90      0.90     14332
```
RDF1 n =100 + nlp


```
                precision    recall  f1-score   support

    Negative       0.91      0.92      0.92      4297
    Positive       0.85      0.94      0.89      4009
     Neutral       0.94      0.88      0.91      3592
  Irrelevant       0.96      0.85      0.90      2434

    accuracy                           0.90     14332
   macro avg       0.91      0.90      0.90     14332
weighted avg       0.91      0.90      0.90     14332

```
RDF2 n =200 + nlp


```
               precision    recall  f1-score   support

    Negative       0.91      0.93      0.92      4297
    Positive       0.85      0.94      0.89      4009
     Neutral       0.94      0.88      0.91      3592
  Irrelevant       0.96      0.85      0.90      2434

    accuracy                           0.91     14332
   macro avg       0.91      0.90      0.91     14332
weighted avg       0.91      0.91      0.91     14332
```

RDF3 n =300 + nlp

```
              precision    recall  f1-score   support

    Negative       0.91      0.93      0.92      4297
    Positive       0.85      0.94      0.89      4009
     Neutral       0.94      0.89      0.91      3592
  Irrelevant       0.96      0.85      0.90      2434

    accuracy                           0.91     14332
   macro avg       0.92      0.90      0.91     14332
weighted avg       0.91      0.91      0.91     14332
```





# validate

In [ ]:
test = pd.read_csv('/content/twitter_validation.csv',header=None)

In [ ]:
test = test.rename(columns={0: "Id",1:"entity",2:"sentiment",3:"Tweet_content"})

In [ ]:
vtest=test['new_sent']

In [ ]:
pred = clf.predict(test)

In [ ]:
pred